In [37]:
%matplotlib inline
#import seaborn; seaborn.set()
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [3]:
data = pd.read_csv('weatherAUS.csv')

In [4]:
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 24 columns):
Date             142193 non-null object
Location         142193 non-null object
MinTemp          141556 non-null float64
MaxTemp          141871 non-null float64
Rainfall         140787 non-null float64
Evaporation      81350 non-null float64
Sunshine         74377 non-null float64
WindGustDir      132863 non-null object
WindGustSpeed    132923 non-null float64
WindDir9am       132180 non-null object
WindDir3pm       138415 non-null object
WindSpeed9am     140845 non-null float64
WindSpeed3pm     139563 non-null float64
Humidity9am      140419 non-null float64
Humidity3pm      138583 non-null float64
Pressure9am      128179 non-null float64
Pressure3pm      128212 non-null float64
Cloud9am         88536 non-null float64
Cloud3pm         85099 non-null float64
Temp9am          141289 non-null float64
Temp3pm          139467 non-null float64
RainToday        140787 non-null obje

In [6]:
data['Location'] = data['Location'].astype('category')
data['WindGustDir'] = data['WindGustDir'].astype('category')
data['WindDir9am'] = data['WindDir9am'].astype('category')
data['WindDir3pm'] = data['WindDir3pm'].astype('category')
data['RainToday'] = data['RainToday'].astype('category')

In [7]:
data = data.drop(['Evaporation','Sunshine','Cloud9am','Cloud3pm'],axis = 1)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 20 columns):
Date             142193 non-null object
Location         142193 non-null category
MinTemp          141556 non-null float64
MaxTemp          141871 non-null float64
Rainfall         140787 non-null float64
WindGustDir      132863 non-null category
WindGustSpeed    132923 non-null float64
WindDir9am       132180 non-null category
WindDir3pm       138415 non-null category
WindSpeed9am     140845 non-null float64
WindSpeed3pm     139563 non-null float64
Humidity9am      140419 non-null float64
Humidity3pm      138583 non-null float64
Pressure9am      128179 non-null float64
Pressure3pm      128212 non-null float64
Temp9am          141289 non-null float64
Temp3pm          139467 non-null float64
RainToday        140787 non-null category
RISK_MM          142193 non-null float64
RainTomorrow     142193 non-null object
dtypes: category(5), float64(13), object(2)
memory usage: 17.0+ MB

In [9]:
data.describe()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RISK_MM
count,141556.000000,141871.000000,140787.000000,132923.000000,140845.000000,139563.000000,140419.000000,138583.000000,128179.000000,128212.000000,141289.000000,139467.000000,142193.000000
mean,12.186400,23.226784,2.349974,39.984292,14.001988,18.637576,68.843810,51.482606,1017.653758,1015.258204,16.987509,21.687235,2.360682
std,6.403283,7.117618,8.465173,13.588801,8.893337,8.803345,19.051293,20.797772,7.105476,7.036677,6.492838,6.937594,8.477969
min,-8.500000,-4.800000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.500000,977.100000,-7.200000,-5.400000,0.000000
25%,7.600000,17.900000,0.000000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.900000,1010.400000,12.300000,16.600000,0.000000
50%,12.000000,22.600000,0.000000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.600000,1015.200000,16.700000,21.100000,0.000000
75%,16.800000,28.200000,0.800000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.400000,1020.000000,21.600000,26.400000,0.800000
max,33.900000,48.100000,371.000000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.000000,1039.600000,40.200000,46.700000,371.000000


In [10]:
data.corr()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RISK_MM
MinTemp,1.000000,0.736267,0.104255,0.177285,0.176005,0.175749,-0.234211,0.005999,-0.451260,-0.461623,0.901813,0.708865,0.124743
MaxTemp,0.736267,1.000000,-0.074839,0.067690,0.014680,0.050800,-0.505432,-0.509270,-0.332293,-0.427279,0.887020,0.984562,-0.044208
Rainfall,0.104255,-0.074839,1.000000,0.133497,0.086816,0.057759,0.223725,0.255312,-0.168085,-0.126728,0.011477,-0.079178,0.308557
WindGustSpeed,0.177285,0.067690,0.133497,1.000000,0.604837,0.686419,-0.215461,-0.026663,-0.457891,-0.412922,0.150258,0.032970,0.162923
WindSpeed9am,0.176005,0.014680,0.086816,0.604837,1.000000,0.519971,-0.270807,-0.031607,-0.227923,-0.174916,0.129298,0.005108,0.069404
WindSpeed3pm,0.175749,0.050800,0.057759,0.686419,0.519971,1.000000,-0.145942,0.015903,-0.295567,-0.254988,0.163601,0.028567,0.049240
Humidity9am,-0.234211,-0.505432,0.223725,-0.215461,-0.270807,-0.145942,1.000000,0.667388,0.139519,0.186955,-0.472826,-0.499777,0.172417
Humidity3pm,0.005999,-0.509270,0.255312,-0.026663,-0.031607,0.015903,0.667388,1.000000,-0.027449,0.051840,-0.221467,-0.557989,0.313183
Pressure9am,-0.451260,-0.332293,-0.168085,-0.457891,-0.227923,-0.295567,0.139519,-0.027449,1.000000,0.961348,-0.422773,-0.287301,-0.163673
Pressure3pm,-0.461623,-0.427279,-0.126728,-0.412922,-0.174916,-0.254988,0.186955,0.051840,0.961348,1.000000,-0.470325,-0.389863,-0.164184


In [11]:
data = data.drop(['MinTemp','Temp9am','Temp3pm','Pressure9am'],axis=1)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 16 columns):
Date             142193 non-null object
Location         142193 non-null category
MaxTemp          141871 non-null float64
Rainfall         140787 non-null float64
WindGustDir      132863 non-null category
WindGustSpeed    132923 non-null float64
WindDir9am       132180 non-null category
WindDir3pm       138415 non-null category
WindSpeed9am     140845 non-null float64
WindSpeed3pm     139563 non-null float64
Humidity9am      140419 non-null float64
Humidity3pm      138583 non-null float64
Pressure3pm      128212 non-null float64
RainToday        140787 non-null category
RISK_MM          142193 non-null float64
RainTomorrow     142193 non-null object
dtypes: category(5), float64(9), object(2)
memory usage: 12.6+ MB


In [13]:
data[['Year','Month','Day']] = data['Date'].str.split('-',expand=True,)
data['Year'] = data['Year'].astype('category')
data['Month'] = data['Month'].astype('category')
data['Day'] = data['Day'].astype('category')

In [14]:
data = data.drop('Date',axis=1)

In [15]:
data.head()

,Location,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure3pm,RainToday,RISK_MM,RainTomorrow,Year,Month,Day
0,Albury,22.9,0.6,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.1,No,0.0,No,2008,12,01
1,Albury,25.1,0.0,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1007.8,No,0.0,No,2008,12,02
2,Albury,25.7,0.0,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1008.7,No,0.0,No,2008,12,03
3,Albury,28.0,0.0,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1012.8,No,1.0,No,2008,12,04
4,Albury,32.3,1.0,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1006.0,No,0.2,No,2008,12,05


In [16]:
data.loc[data['RainTomorrow'] == 'No','RainTomorrow'] = 0
data.loc[data['RainTomorrow'] == 'Yes','RainTomorrow'] = 1

In [17]:
target = data['RainTomorrow']
data_transformed = data.drop('RainTomorrow',axis=1)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(data_transformed, target, test_size=0.33, random_state=42)
num_attr = []
cat_attr = []
for i in X_train.columns:
    if X_train[i].dtypes == 'float64':
        num_attr.append(i)
    else:
        cat_attr.append(i)
print(num_attr)
print(cat_attr)

['MaxTemp', 'Rainfall', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure3pm', 'RISK_MM']
['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'Year', 'Month', 'Day']


In [19]:
num_pipeline = Pipeline([('imputer',SimpleImputer(strategy='mean')),('normalizer',Normalizer())])
cat_pipeline = Pipeline([('imputer',SimpleImputer(strategy='most_frequent')),('onehot',OneHotEncoder())])
full_pipeline = ColumnTransformer([('num',num_pipeline,num_attr),('cat',cat_pipeline,cat_attr)])

X_train_transformed = full_pipeline.fit_transform(X_train)
X_test_tr = full_pipeline.transform(X_test)

In [40]:
clf =  RandomForestClassifier(n_estimators=100,max_leaf_nodes=16, max_depth=5,class_weight='balanced', random_state=0)
clf.fit(X_train_transformed, y_train) 


y_pred = clf.predict(X_test_tr)
print(cross_val_score(clf, X_test_tr, y_test, cv=3, scoring="accuracy"))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[ 0.94521161  0.94782942  0.94367368]
[[34278  2188]
 [  586  9872]]
              precision    recall  f1-score   support

           0       0.98      0.94      0.96     36466
           1       0.82      0.94      0.88     10458

    accuracy                           0.94     46924
   macro avg       0.90      0.94      0.92     46924
weighted avg       0.95      0.94      0.94     46924

0.940883130168


In [41]:
clf = DecisionTreeClassifier(random_state = 11, class_weight = "balanced",max_depth = 5)
clf.fit(X_train_transformed,y_train)

y_pred = clf.predict(X_test_tr)
print(confusion_matrix(y_test, y_pred))
print(cross_val_score(clf, X_test_tr, y_test, cv=3, scoring="accuracy"))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[36466     0]
 [    0 10458]]
[ 1.  1.  1.]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     36466
           1       1.00      1.00      1.00     10458

    accuracy                           1.00     46924
   macro avg       1.00      1.00      1.00     46924
weighted avg       1.00      1.00      1.00     46924

1.0
